In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [23]:
urls = ["https://app.hirenimble.com/api/marketplace/list/districts/?state_abbreviation=TN",
        "https://app.hirenimble.com/api/marketplace/list/schools/?state_abbreviation=TN"]

for i in np.arange(1,24,1):
    page_url = f"https://app.hirenimble.com/api/marketplace/schools/?state_abbreviation=TN&page={i}"
    urls.append(page_url)

data = []

for url in urls:
    response = requests.get(url)
    data.append(response.json())

In [26]:
districts = data[0]
schools = data[1]
jobs = data[2:24]

In [28]:
len(schools)

1906

In [31]:
district_df = pd.DataFrame(districts)
#districts.rename(columns= {0: 'district'})
district_df

,name,nces_id
0,Lauderdale County,4702310
1,Robertson County,4703600
2,Arlington,4700152
3,Collierville,4700149
4,Bartlett,4700153
...,...,...
143,Achievement School District,4700147
144,Haywood County,4701770
145,Johnson County,4702160
146,Lebanon,4702370


In [33]:

school_info = pd.DataFrame()

names = []
nces_ids = []
addresses = []
zips = []
districts = []
salary_links = []
grades = []
school_types = []
student_counts = []
principal_names = []

for n in range(len(jobs)):
    for school in jobs[n]['items']:
        name = school['name']
        nces_id = school['nces_id']
        address = school['physical_address']['line_1']
        zip = school['physical_address']['zipcode']
        district = school['district']['name']
        salary_link = school['district']['salary_schedule_link']
        grades_list = list(school['grades'])
        school_type = school['school_metadata']['school_type']
        student_count = school['school_metadata']['student_count']
#        principal_name = school['principal']['first_name'] + " " + school['principal']['last_name']
        
        names.append(name)
        nces_ids.append(nces_id)
        addresses.append(address)
        zips.append(zip)
        districts.append(district)
        salary_links.append(salary_link)
        grades.append(grades_list)
        school_types.append(school_type)
        student_counts.append(student_count)
#        principal_names.append(principal_name)


school_info['school'] = names
school_info['nces_id'] = nces_ids
school_info['address'] = addresses
school_info['zip'] = zips
school_info['district'] = districts
school_info['grades'] = grades
school_info['school_type'] = school_types
school_info['student_count'] = student_counts
school_info['salary_link'] = salary_links
#schools['principal_name'] = principal_names


school_levels = []

for grades_list in school_info['grades']:
    
    school_level = []
    
    if any(grade in grades_list for grade in ['1','2','3','4']):
        school_level.append('Elementary')
        
    if '7' in grades_list:
        school_level.append('Middle')
        
    if any(grade in grades_list for grade in ['10','11','12']):
        school_level.append('High')
    
    school_levels.append(school_level)

school_info['school_type2'] = school_levels


school_info

,school,nces_id,address,zip,district,grades,school_type,student_count,salary_link,school_type2
0,John Trotwood Moore Middle,470318001346,4425 Granny White Pike,37204,Metro Nashville Public Schools,"[6, 7, 8]",Public,528.0,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,[Middle]
1,Explore Community School,470318002449,707 S 7th St,37206,Metro Nashville Public Schools,"[1, 2, 3, 4, 5, 6, 7, 8, KG]",Public Charter,535.0,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,"[Elementary, Middle]"
2,Bellshire Elementary,470318001262,1128 Bell Grimes Ln,37207,Metro Nashville Public Schools,"[1, 2, 3, 4, 5, KG, PK]",Public,402.0,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,[Elementary]
3,Smyrna Primary,470369001530,200 Walnut St,37167,Rutherford County,"[1, 2, 3, 4, 5, KG, PK]",Public,581.0,https://drive.google.com/file/d/1tzQ_-LKsypULx...,[Elementary]
4,Stewartsboro Elementary,470369000115,10479 Old Nashville Hwy,37167,Rutherford County,"[1, 2, 3, 4, 5, KG, PK]",Public,718.0,https://drive.google.com/file/d/1tzQ_-LKsypULx...,[Elementary]
...,...,...,...,...,...,...,...,...,...,...
1095,Friendship Elementary,470085001898,6117 Highway 189,38034,Crockett County,"[1, 2, 3, 4, 5, KG, PK]",Public,115.0,https://drive.google.com/file/d/1pelY_IUCkd8zl...,[Elementary]
1096,Maury City Elementary,470085001902,5442 Highway 88,38050,Crockett County,"[1, 2, 3, 4, 5, KG, PK]",Public,230.0,https://drive.google.com/file/d/1pelY_IUCkd8zl...,[Elementary]
1097,New Directions Academy,470102000327,4000 Highway 48 North,37036,Dickson County,"[10, 11, 12, 6, 7, 8, 9]",Public,18.0,https://drive.google.com/file/d/1VyhsFQOvOYW63...,"[Middle, High]"
1098,Charlotte Middle School,470102000328,250 Humphries St,37036,Dickson County,"[6, 7, 8]",Public,436.0,https://drive.google.com/file/d/1VyhsFQOvOYW63...,[Middle]


In [35]:
schools_df = pd.DataFrame(schools)

cities = []
for address in schools_df.physical_address:
    city = address['city']
    cities.append(city)
    
schools_df.drop(columns='physical_address', inplace=True)
schools_df['city'] = cities
schools_df = schools_df.rename(columns={'name':'school'})
schools_df

,school,nces_id,city
0,Yorkville Elementary,470140000423,Yorkville
1,Tennessee Connections Academy Johnson County 9-12,470216002622,White House
2,Ocoee Middle School,470033000089,Cleveland
3,Trezevant High,470014801166,Memphis
4,Daniel Boone High School,470438001756,Gray
...,...,...,...
1901,Woodland Park Elementary,470450001980,Sparta
1902,Woodlawn Elementary,470303001222,Woodlawn
1903,Woodstock Middle School,470014801595,Millington
1904,Wright Middle,470318001389,Nashville


In [37]:
schools_df = schools_df.merge(school_info, how='outer', on='nces_id')

In [39]:
schools_df[(schools_df.school.duplicated() == True) & (schools_df.district.duplicated() == True)]

AttributeError: 'DataFrame' object has no attribute 'school'

In [41]:
schools_df = schools_df.drop(columns='school_y')
schools_df

,school_x,nces_id,city,address,zip,district,grades,school_type,student_count,salary_link,school_type2
0,Alpha Elementary,470000100477,Morristown,5620 Old Us 11e Hwy,37814,Hamblen County,"[1, 2, 3, 4, 5, KG]",Public,606.0,https://drive.google.com/file/d/1gOmzE-qHfhDrA...,[Elementary]
1,East Ridge Middle School,470000100479,Whitesburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fairview Marguerite,470000100480,Morristown,2125 Fairview Rd,37814,Hamblen County,"[1, 2, 3, 4, 5, KG, PK]",Public,490.0,https://drive.google.com/file/d/1gOmzE-qHfhDrA...,[Elementary]
3,Manley Elementary,470000100481,Morristown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Russellville Elementary,470000100483,Russellville,5655 Old Russellville Pike,37860,Hamblen County,"[1, 2, 3, 4, 5, KG, PK]",Public,624.0,https://drive.google.com/file/d/1gOmzE-qHfhDrA...,[Elementary]
...,...,...,...,...,...,...,...,...,...,...,...
1901,Springdale Elementary School,470455002479,Mt. Juliet,5675 Central Pike,37122,Wilson County,"[1, 2, 3, 4, 5, KG, PK]",Public,823.0,https://drive.google.com/file/d/1Fsi53MEKr4-P_...,[Elementary]
1902,Gladeville Middle School,470455002593,Mt. Juliet,8275 Stewarts Ferry Pike,37122,Wilson County,"[6, 7, 8]",Public,1008.0,https://drive.google.com/file/d/1Fsi53MEKr4-P_...,[Middle]
1903,Green Hill High School,470455002639,MT. JULIET,220 NORTH GREENHILL Rd,37122,Wilson County,"[10, 11, 12, 9]",Public,1662.0,https://drive.google.com/file/d/1Fsi53MEKr4-P_...,[High]
1904,Barry Tatum Academy,470455002686,Lebanon,205 Stumpy Ln,37090,Wilson County,"[10, 11, 12, 6, 7, 8, 9]",Public,46.0,https://drive.google.com/file/d/1Fsi53MEKr4-P_...,"[Middle, High]"


In [43]:
schools_df[['district','salary_link']].drop_duplicates().sort_values('salary_link').head(15)

,district,salary_link
1067,Madison County,https://drive.google.com/file/d/1-IkuhRs5X7RE5...
1738,Trousdale County,https://drive.google.com/file/d/10zXYMWozwAnQ1...
512,Coffee County,https://drive.google.com/file/d/14KjH-KzO_e_8x...
1430,Obion County,https://drive.google.com/file/d/14SGUeZTocc7zM...
1422,Oak Ridge,https://drive.google.com/file/d/17XJIr4AtljT4Z...
1029,Lenoir City,https://drive.google.com/file/d/18IzxY4fmKGcyP...
64,Memphis-Shelby County Schools,https://drive.google.com/file/d/18aiSDErNckscd...
1259,Metro Nashville Public Schools,https://drive.google.com/file/d/1A4h4YNjOMlL-D...
531,Cumberland County,https://drive.google.com/file/d/1CXwnPchnIlFgQ...
1742,Tullahoma,https://drive.google.com/file/d/1Cn9qpHD8fbb50...


In [45]:
schools_df.to_csv('schools_df.csv', index=False)

In [47]:
jobs_df = pd.DataFrame()

job_names = []
job_schools = []
nces_ids = []
job_districts = []
categories = []
posting_types = []
salary_links = []

for n in range(len(jobs)):
    for school in jobs[n]['items']:
        job_school = school['name']
        job_district = school['district']['name']
        nces_id = school['nces_id']
        salary_link = school['district']['salary_schedule_link']
        
        for job in school['jobpostings']:
            job_name = job['title']
            category = job['subcategories'][0]
            posting_type = job['pooled_posting_type']

            job_names.append(job_name)
            job_districts.append(job_district)
            categories.append(category)
            salary_links.append(salary_link)
            
            if posting_type == 'single_posting':
                job_schools.append(job_school)
                nces_ids.append(nces_id)
                posting_types.append('School')
            
            elif posting_type == 'pooled_posting':
                job_schools.append(np.nan)
                nces_ids.append(np.nan)
                posting_types.append('District')

# create jobs_df dataframe

jobs_df['name'] = job_names
jobs_df['school'] = job_schools
jobs_df['nces_id'] = nces_ids
jobs_df['district'] = job_districts
jobs_df['category'] = categories
jobs_df['posting_type'] = posting_types
jobs_df['salary_link'] = salary_links

jobs_df

jobs_df = jobs_df.merge(schools_df, on=['nces_id', 'salary_link', 'district'], how='left')

jobs_df = jobs_df.drop(columns='school_x')

#len(set(jobs_df.school))
#len(schools_df)
jobs_df

,name,school,nces_id,district,category,posting_type,salary_link,city,address,zip,grades,school_type,student_count,school_type2
0,Therapist Ex Ed Speech Language Pool (2025-202...,NaN,NaN,Metro Nashville Public Schools,Special Education (General),District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Teacher Language Spanish (2025-2026) - (25002522),John Trotwood Moore Middle,470318001346,Metro Nashville Public Schools,Foreign Language: Spanish,School,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,Nashville,4425 Granny White Pike,37204,"[6, 7, 8]",Public,528.0,[Middle]
2,Therapist Ex Ed Speech Language (2025-2026) - ...,NaN,NaN,Metro Nashville Public Schools,Special Education (General),District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spec Instructional (Pool 2025-26 SY) - (25001627),NaN,NaN,Metro Nashville Public Schools,Reading/Literacy,District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Teacher Health Physical Education Secondary (2...,NaN,NaN,Metro Nashville Public Schools,Health & Physical Education,District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4845,Middle School Math Teacher Grades 6-8 (2025-20...,NaN,NaN,Dickson County,Mathematics,District,https://drive.google.com/file/d/1VyhsFQOvOYW63...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4846,Vision Teacher (Part-time),NaN,NaN,Dickson County,Special Education (Comprehensive),District,https://drive.google.com/file/d/1VyhsFQOvOYW63...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4847,Middle School Math Teacher Grades 6-8 (2025-20...,NaN,NaN,Dickson County,Mathematics,District,https://drive.google.com/file/d/1VyhsFQOvOYW63...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4848,Vision Teacher (Part-time),NaN,NaN,Dickson County,Special Education (Comprehensive),District,https://drive.google.com/file/d/1VyhsFQOvOYW63...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
jobs_df = jobs_df.drop_duplicates(subset=['name', 'district'], keep='first', inplace=False)

In [102]:
jobs_df.to_csv('jobs_df.csv', index=False)

In [104]:
jobs_df

,name,school,nces_id,district,category,posting_type,salary_link,city,address,zip,grades,school_type,student_count,school_type2,county,salary,fips
0,Therapist Ex Ed Speech Language Pool (2025-202...,NaN,NaN,Metro Nashville Public Schools,Special Education (General),District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Davidson,70137,47037
1,Teacher Language Spanish (2025-2026) - (25002522),John Trotwood Moore Middle,470318001346,Metro Nashville Public Schools,Foreign Language: Spanish,School,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,Nashville,4425 Granny White Pike,37204,"[6, 7, 8]",Public,528.0,[Middle],Davidson,70137,47037
2,Therapist Ex Ed Speech Language (2025-2026) - ...,NaN,NaN,Metro Nashville Public Schools,Special Education (General),District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Davidson,70137,47037
3,Spec Instructional (Pool 2025-26 SY) - (25001627),NaN,NaN,Metro Nashville Public Schools,Reading/Literacy,District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Davidson,70137,47037
4,Teacher Health Physical Education Secondary (2...,NaN,NaN,Metro Nashville Public Schools,Health & Physical Education,District,https://drive.google.com/file/d/1A4h4YNjOMlL-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Davidson,70137,47037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,Math Teacher(2025-2026),Coffee County Central High School,470078000285,Coffee County,Mathematics,School,https://drive.google.com/file/d/14KjH-KzO_e_8x...,Manchester,100 Red Raider Dr,37355,"[10, 11, 12]",Public,1285.0,[High],Coffee,56686,47031
851,Driver Education Teacher,Coffee County Central High School,470078000285,Coffee County,Career and Technical Education,School,https://drive.google.com/file/d/14KjH-KzO_e_8x...,Manchester,100 Red Raider Dr,37355,"[10, 11, 12]",Public,1285.0,[High],Coffee,56686,47031
852,Elementary School Teacher,Friendship Elementary,470085001898,Crockett County,Elementary/Multiple Subjects,School,https://drive.google.com/file/d/1pelY_IUCkd8zl...,Friendship,6117 Highway 189,38034,"[1, 2, 3, 4, 5, KG, PK]",Public,115.0,[Elementary],Crockett,49559,47033
853,Future Grades Prek-5 Certified Teacher,NaN,NaN,Crockett County,Elementary/Multiple Subjects,District,https://drive.google.com/file/d/1pelY_IUCkd8zl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crockett,49559,47033


In [84]:
jobs_df_updated.to_csv('jobs_df_updated.csv', index=False)

In [86]:
jobs_df.salary_link.value_counts()

salary_link
https://drive.google.com/file/d/1A4h4YNjOMlL-DpqEOltTxSK3dyWsDIrF/view                162
https://drive.google.com/file/d/1ib4nc3LBIojVV91W5u2irw28cZ4uHD6H/view                 46
https://drive.google.com/file/d/1tD8aivGHZ5LSj4imSgiaH6rS0h1e7Y4T/view                 41
https://drive.google.com/file/d/1tzQ_-LKsypULxy-UYWeMRcsUXhapl-3n/view                 35
https://drive.google.com/file/d/1ixwnBozzpke1bRmAX1P_nulpwvAduKoO/view                 29
https://drive.google.com/file/d/1Fsi53MEKr4-P_JY2gSVMq97BN1oGBLNA/view                 29
https://drive.google.com/file/d/1f2GeoM9UjVlk2yTf9T0kgpdCHJ6rMXKx/view                 27
https://drive.google.com/file/d/1gOmzE-qHfhDrALaZ7fyOf0HUXMvrVmVy/view                 24
https://drive.google.com/file/d/1YgPzOsmX82-wSvC89LJzhsj0J-s-OrSu/view                 24
https://www.roaneschools.com/page/salary-tables                                        21
https://drive.google.com/file/d/1cq1tUzcNcdayP0hKV3C0_nkqg9KEoDpP/view                 1

In [88]:
set(jobs_df.district)

{'Anderson County',
 'Bartlett',
 'Bedford County',
 'Bledsoe County',
 'Blount County',
 'Bradley County',
 'Campbell County',
 'Cheatham County',
 'Chester County',
 'Cleveland',
 'Cocke County',
 'Coffee County',
 'Collierville',
 'Crockett County',
 'Dickson County',
 'Dyer County',
 'Fayette County Public Schools',
 'Franklin Special School District',
 'Germantown',
 'Greene County',
 'Grundy County',
 'Hamblen County',
 'Haywood County',
 'Henry County',
 'Humboldt City Schools',
 'Jefferson County',
 'Johnson City',
 'Johnson County',
 'Kingsport',
 'Knox County',
 'Lakeland',
 'Lebanon',
 'Lenoir City',
 'Macon County',
 'Madison County',
 'Marshall County',
 'Maryville',
 'Maury County',
 'Memphis-Shelby County Schools',
 'Metro Nashville Public Schools',
 'Milan',
 'Morgan County',
 'Murfreesboro',
 'Oak Ridge',
 'Putnam County',
 'Roane County',
 'Robertson County',
 'Rutherford County',
 'Sequatchie County',
 'Sumner County',
 'Tipton County',
 'Trenton',
 'Trousdale County

In [90]:
school_salaries = pd.read_csv('school_salaries - Sheet4.csv')
fips_codes = pd.read_csv('school_salaries - fips codes.csv')

In [92]:
fips_codes = fips_codes.rename(columns = {'Unnamed: 1': 'county'})

In [94]:
fips_codes.fips = fips_codes.fips.str[3:]

In [96]:
school_salaries = school_salaries.rename(columns={'District Name': 'district', 'County': 'county', 'Avg Salary': 'salary'})

In [98]:
school_salaries = school_salaries.merge(fips_codes)

In [99]:
jobs_df = jobs_df.merge(school_salaries)

In [78]:
fips_codes

,fips,county
0,47001,Anderson
1,47003,Bedford
2,47005,Benton
3,47007,Bledsoe
4,47009,Blount
...,...,...
90,47181,Wayne
91,47183,Weakley
92,47185,White
93,47187,Williamson


In [29]:
import gdown

In [83]:
files_retrieved=['1RGzyaL3Rc2JuObVIsJIxUrna268dRDMB',
'1x7Uzhimaj7i8jscBdzJxgvXUv6MhKgpq',
'1DoyRPIa0toz-T8rCZNwlTPq8nO52UQ3d',
'1f2GeoM9UjVlk2yTf9T0kgpdCHJ6rMXKx',
'1Y2Emg9htabsqHSZxPF2YP2QXjUGmzLq4',
'1RyMp_qqGnnGUFG6hX_5Xu6u-767tmvcd',
'1Cn9qpHD8fbb50TYoCVqp36ztLe0Bs2lD',
'1vTkRKqtJy0wpo0dE9z1oC2c6B-CoJraV',
'1jeKPBi4rkWNm8pRAHtzvpOxs_wE27Lsf',
'1qO45CVy7bgbUgXNazZqVOfAwZgCm1JHS',
'1ib4nc3LBIojVV91W5u2irw28cZ4uHD6H',
'18aiSDErNckscdPF6a5CdCDayEkkfBf94',
'1tzQ_-LKsypULxy-UYWeMRcsUXhapl-3n',
'1Dd5STmPN07IYbphhF_zduSXc3GH_7jlu',
'1XKsqvlVjU85UrEfV5wh_uusAWjPNbSGX',
'1PQiC7Y40tuM9ibPNdcq7i7lT54rYgPSJ',
'1rSH3k8Llp1DJmkgUSZAsLfaNK1W2RHJD',
'17XJIr4AtljT4Z50kXSvkMh8epmPTCQ2P',
'1EBEE_Rf0mAD5I4oKUmvli7lw2-SOhdKM',
'1wVeMcIYqTowlF-Vf5W5NDUcKKVxMY8WN',
'1ts1vHFd82qgt8RNmJb5ffxyYGtimh35w',
'1ixwnBozzpke1bRmAX1P_nulpwvAduKoO',
'1HfG9OzsrrkZbRmK0Yi2dU6-LPexgLvGw',
'1X5TRVgEPIeMhpXz-v5VkPfOJeeaD8CMC',
'1-IkuhRs5X7RE5k0z2SruZAE5iPiitm4w',
'1od0egZ7trlVJJQgwjxn46WySkNW9GcQO',
'1q9ihHImI4nJ8uV7vLpyvv8X7DpK5FrZt',
'1zCHLTSj_lUDSApYjmSzbM1XacGXbyjwM',
'18IzxY4fmKGcyPdtt11juIFPRETR77Jr0',
'1tD8aivGHZ5LSj4imSgiaH6rS0h1e7Y4T',
'1Vb9tKZhORpB-JHjgf6FNjfc0myxumGwR',
'1wKNH6zexmRAATuvaZRdKLBxQqkKNh8-W',
'1O49KdBvJMWJL6kl8RXGZaWW-rt7WpDZh',
'1vM4CFkZBcDloS0YKcNBmkNyNwV3vNusW',
'1j_huFc8jfwMTV0Kk-_XXru4b9Pqaz_ta',
'1W2FMLAqq3xAwlry82abCZwVRABY9ERfb',
'1gOmzE-qHfhDrALaZ7fyOf0HUXMvrVmVy',
'1XcDuPjXgoSvTi62ZaG8kjeh3dZKk878o',
'1kZv5LZzPRpE3NIofQqelLgLG1oi9_yuV',
'1oqqfArsRliH1dIyBJP8brUKHfuoJ_cJ2',
'1VGAmY9_ApqcBH89FqSksY-BaNsCS5vkD',
'1fesD2Sv0uhq7owg-Pzi6vtLXmiBl9kaC',
'1LcOUG1hHrUqu9fYL79cArYl1o0YVOtkx',
'1R38qhWNLPj8czX0MnTbJ8BtbynxJwEoJ',
'1ktrObvISrckQw33F64xliqUqRU0rHOPe',
'1VyhsFQOvOYW63_ULMKu6FynMvz-Yzsqb',
'1A4h4YNjOMlL-DpqEOltTxSK3dyWsDIrF',
'1pelY_IUCkd8zlZI57TY2gosnmV0z7bRo',
'1rInZakiIV5ep8ANCMNk3pMIsiipWuBjJ',
'14KjH-KzO_e_8xJpTQw6pdalndT6mo63w',
'1iB_O1iMSHvdidpQHf15FS08nM2c2sTCp',
'1K-RjFHOsa35pJUKXZxXDFH7mf39kGljK',
'1ZlM6bk3t5GwfcrvmO-WDknCtBnP90BSQ',
'1YgPzOsmX82-wSvC89LJzhsj0J-s-OrSu',
'1n8PZrwuPQnv-hx9MslKusZGw_YogLw3n',
'1Ef2d54N6AVdEg90xEQ0SQWcsyIGlzuWj',
'1lh1oQsKqpoJ6M396QhRoIxU-EiE-Zb8A',
'1zfyYzIeuAjdi-1i7Xs2whvDgKNjbdlpT',
'1QXKjC54z-ASDzZIyQvAjx2xCd6L8IQf5',
'1cq1tUzcNcdayP0hKV3C0_nkqg9KEoDpP',
'1ZWDp-CQMzYibK2d12vWW8JaNjzeR6cGS',
'1KYG9WFyK8Z4bQnQcX1W2BBlIIBdjXAsO']

In [85]:
files_needed = []

for file in jobs_df.salary_link.unique():
    if pd.notna(file):
        file_id = file[32:-5]
        if file_id not in files_retrieved:
            files_needed.append(file_id)

files_needed

['e/salary-t', '1riDcOZgImUilQYf2DfDlK134V1-RS0VX']

In [77]:
file_ids = []
#bad_files = []

for file in jobs_df.salary_link.unique():
    if pd.notna(file):
        file_id = file[32:-5]
        if file_id not in files_retrieved:
            file_ids.append(file_id)

len(file_ids)

28

In [71]:
urls = [f"https://drive.google.com/uc?id={id}" for id in file_ids]
urls

['https://drive.google.com/uc?id=1RGzyaL3Rc2JuObVIsJIxUrna268dRDMB',
 'https://drive.google.com/uc?id=1W2FMLAqq3xAwlry82abCZwVRABY9ERfb',
 'https://drive.google.com/uc?id=1rInZakiIV5ep8ANCMNk3pMIsiipWuBjJ',
 'https://drive.google.com/uc?id=e/salary-t',
 'https://drive.google.com/uc?id=1jeKPBi4rkWNm8pRAHtzvpOxs_wE27Lsf',
 'https://drive.google.com/uc?id=1vTkRKqtJy0wpo0dE9z1oC2c6B-CoJraV',
 'https://drive.google.com/uc?id=1RyMp_qqGnnGUFG6hX_5Xu6u-767tmvcd',
 'https://drive.google.com/uc?id=1KYG9WFyK8Z4bQnQcX1W2BBlIIBdjXAsO',
 'https://drive.google.com/uc?id=1ZlM6bk3t5GwfcrvmO-WDknCtBnP90BSQ',
 'https://drive.google.com/uc?id=1LcOUG1hHrUqu9fYL79cArYl1o0YVOtkx',
 'https://drive.google.com/uc?id=1XcDuPjXgoSvTi62ZaG8kjeh3dZKk878o',
 'https://drive.google.com/uc?id=1Vb9tKZhORpB-JHjgf6FNjfc0myxumGwR',
 'https://drive.google.com/uc?id=1wKNH6zexmRAATuvaZRdKLBxQqkKNh8-W',
 'https://drive.google.com/uc?id=18IzxY4fmKGcyPdtt11juIFPRETR77Jr0',
 'https://drive.google.com/uc?id=1X5TRVgEPIeMhpXz-v5VkPfO

In [89]:
gdown.download('https://drive.google.com/uc?id=1riDcOZgImUilQYf2DfDlK134V1-RS0VX', output=url.split('=')[-1] + ".pdf", quiet=False)

FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1riDcOZgImUilQYf2DfDlK134V1-RS0VX

but Gdown can't. Please check connections and permissions.

In [79]:
for url in urls:
    if url:
        try:
            gdown.download(url, output=url.split('=')[-1] + ".pdf", quiet=False)
        except:
            bad_files.append(url)

Downloading...
From: https://drive.google.com/uc?id=1RGzyaL3Rc2JuObVIsJIxUrna268dRDMB
To: /Users/gretagoss/Downloads/DATA_304/test/data-304/data/final_project/1RGzyaL3Rc2JuObVIsJIxUrna268dRDMB.pdf
100%|██████████| 97.0k/97.0k [00:00<00:00, 76.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1W2FMLAqq3xAwlry82abCZwVRABY9ERfb
To: /Users/gretagoss/Downloads/DATA_304/test/data-304/data/final_project/1W2FMLAqq3xAwlry82abCZwVRABY9ERfb.pdf
100%|██████████| 48.9k/48.9k [00:00<00:00, 2.38MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rInZakiIV5ep8ANCMNk3pMIsiipWuBjJ
To: /Users/gretagoss/Downloads/DATA_304/test/data-304/data/final_project/1rInZakiIV5ep8ANCMNk3pMIsiipWuBjJ.pdf
100%|██████████| 49.6k/49.6k [00:00<00:00, 4.08MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jeKPBi4rkWNm8pRAHtzvpOxs_wE27Lsf
To: /Users/gretagoss/Downloads/DATA_304/test/data-304/data/final_project/1jeKPBi4rkWNm8pRAHtzvpOxs_wE27Lsf.pdf
100%|██████████| 70.4k/70.4k [00:00<00:00, 3.57MB/s]


In [43]:
file_ids = []
#bad_files = []

for file in jobs_df.salary_link.unique():
    if pd.notna(file):
        file_id = file[32:-5]
        if file_id not in files_retrieved:
            file_ids.append(file_id)

urls = [f"https://drive.google.com/uc?id={id}" for id in file_ids]

for url in urls:
    if url:
        try:
            gdown.download(url, output=url.split('=')[-1] + ".pdf", quiet=False)
        except:
            bad_files.append(url)


'''
file_ids = []
bad_files = []

for file in jobs_df.salary_link.unique()[38:-1]:
    if pd.notna(file):
        file_id = file[32:-5]
        file_ids.append(file_id)

urls = [f"https://drive.google.com/uc?id={id}" for id in file_ids]

for url in urls:
    if url:
        try:
            gdown.download(url, output=url.split('=')[-1] + ".pdf", quiet=False)
        except:
            bad_files.append(url)
'''

In [55]:
sorted(jobs_df.salary_link.unique()[pd.notna(jobs_df.salary_link.unique())])#[37][32:-5]

['https://drive.google.com/file/d/1-IkuhRs5X7RE5k0z2SruZAE5iPiitm4w/view',
 'https://drive.google.com/file/d/14KjH-KzO_e_8xJpTQw6pdalndT6mo63w/view',
 'https://drive.google.com/file/d/17XJIr4AtljT4Z50kXSvkMh8epmPTCQ2P/view',
 'https://drive.google.com/file/d/18IzxY4fmKGcyPdtt11juIFPRETR77Jr0/view',
 'https://drive.google.com/file/d/18aiSDErNckscdPF6a5CdCDayEkkfBf94/view',
 'https://drive.google.com/file/d/1A4h4YNjOMlL-DpqEOltTxSK3dyWsDIrF/view',
 'https://drive.google.com/file/d/1Cn9qpHD8fbb50TYoCVqp36ztLe0Bs2lD/view',
 'https://drive.google.com/file/d/1Dd5STmPN07IYbphhF_zduSXc3GH_7jlu/view',
 'https://drive.google.com/file/d/1DoyRPIa0toz-T8rCZNwlTPq8nO52UQ3d/view',
 'https://drive.google.com/file/d/1EBEE_Rf0mAD5I4oKUmvli7lw2-SOhdKM/view',
 'https://drive.google.com/file/d/1Ef2d54N6AVdEg90xEQ0SQWcsyIGlzuWj/view',
 'https://drive.google.com/file/d/1HfG9OzsrrkZbRmK0Yi2dU6-LPexgLvGw/view',
 'https://drive.google.com/file/d/1K-RjFHOsa35pJUKXZxXDFH7mf39kGljK/view',
 'https://drive.google.co

In [130]:
district_links_needed = []
for district in district_df['name']:
    if district not in set(jobs_df['district']):
        district_links_needed.append(district)

district_links_needed

['Lauderdale County',
 'Arlington',
 'Hamilton County',
 'Clinton',
 'Bristol',
 'Dyersburg',
 'Elizabethton',
 'Hawkins County',
 'Hancock County',
 'Cumberland County',
 'Giles County',
 'Greeneville',
 'Richard City',
 'Sequatchie County',
 'Sevier County',
 'Cannon County',
 'Decatur County',
 'Alcoa',
 'Lawrence County',
 'Sullivan County',
 'Carter County',
 'Unicoi County',
 'Alamo',
 'Athens',
 'Cocke County',
 'Washington County',
 'Bells',
 'Benton County',
 'Bradford',
 'Lexington',
 'Claiborne County',
 'Clay County',
 'Dayton',
 'DeKalb County',
 'Etowah',
 'Fayetteville',
 'Fentress County',
 'Grainger County',
 'Hardin County',
 'Henderson County',
 'Humphreys County',
 'Hickman County',
 'Houston County',
 'Hollow Rock - Bruceton',
 'Huntingdon Special School District',
 'Jackson County',
 'Lincoln County',
 'Lake County',
 'Lewis County',
 'Loudon County',
 'Marion County',
 'McKenzie',
 'McNairy County',
 'Meigs County',
 'Monroe County',
 'Moore County',
 'Oneida Spe

In [169]:
len(set(jobs_df['district']))
len(district_links_needed)

83

In [134]:
from serpapi import GoogleSearch


In [242]:
schools_df.to_csv('schools_df.csv', index = False)
jobs_df.to_csv('jobs_df.csv', index = False)

In [161]:
for district in district_links_needed:

    search = GoogleSearch({
        "q": f"{district} Tennessee teacher salaries", 
        "location": "Tennessee",
        "api_key": "118ec6b7c4e5ddc010bee064d2ec5b7ce4577906547b73cb60d9d8303f1013e2"
      })
    result = search.get_dict()
    print(result["organic_results"][0]["link"])  # Print district website URLs

https://www.lced.net/documents/employee-documents/69480
https://acsk-12.org/employees
https://www.hcde.org/staff/human_resources/salary_schedules
https://www.acs.ac/618120_3
https://www.ziprecruiter.com/Salaries/Teacher-Salary-in-Bristol,TN
https://www.dyersburgcityschools.org/salary
https://www.teacher.org/school-district/elizabethton-city-schools/
https://www.ziprecruiter.com/Salaries/Teacher-Salary-in-Hawkins-County,TN
https://govsalaries.com/salaries/TN/hancock-county-school-district
https://www.ccschools.k12tn.net/salaryinformation
https://www.gcboe.us/employment/2023-2024-salary-schedules
https://www4.gcschools.net/documents/departments/finance/salary-scales/558014
https://govsalaries.com/salaries/TN/richard-city-special-school-district
https://govsalaries.com/salaries/TN/sequatchie-county-school-district
https://www.sevier.org/15724_3
https://www.ccstn.net/apps/pages/index.jsp?uREC_ID=2145681&type=d&pREC_ID=2165809
https://govsalaries.com/salaries/TN/decatur-county-school-distri

KeyError: 'organic_results'

In [155]:
result['organic_results'][0]

{'position': 1,
 'title': 'Bradford Special School District Salaries - Tennessee',
 'link': 'https://govsalaries.com/salaries/TN/bradford-special-school-district',
 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://govsalaries.com/salaries/TN/bradford-special-school-district&ved=2ahUKEwj0p-6GjbeMAxUlGLkGHWkuIPQQFnoECBYQAQ',
 'displayed_link': 'https://govsalaries.com › salaries › bradford-special-scho...',
 'favicon': 'https://serpapi.com/searches/67ebffdb133f3f914e665e61/images/68b85ba633c90a1a6beb2049c33b1dbcb4c942bfc02c1359550ec9b53b7ce4d2.png',
 'snippet': 'Number of employees at Bradford Special School District in year 2020 was 49. Average annual salary was $46,586 and median salary was $45,001. Bradford Special ...',
 'snippet_highlighted_words': ['Average annual salary was $46,586'],
 'source': 'GovSalaries'}